In [14]:

# CSV 파일 경로 지정
csv_file_path = '/Users/jhkim97/Downloads/translated-activity-output.csv'

# CSV 파일 읽어오기
df = pd.read_csv(csv_file_path)

# 정제된 데이터를 저장할 새로운 DataFrame 생성
cleaned_df = df.copy()

# 정규 표현식을 사용하여 특수문자 제거
def clean_text(text):
    if isinstance(text, str):
        # 영어, 숫자, 한글, 공백을 제외한 모든 문자를 제거
        cleaned_text = re.sub(r'[^a-zA-Z0-9가-힣\s]', '', text)
        return cleaned_text
    else:
        # 문자열이 아닌 경우 그대로 반환
        return text

# 모든 셀에 대해 특수문자 제거 함수를 적용
for column in cleaned_df.columns:
    cleaned_df[column] = cleaned_df[column].apply(clean_text)

In [18]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from scipy.spatial.distance import cosine
import numpy as np

top_10_similarities = []

# activity_output.csv 파일에서 데이터 읽어오기
# 가정: cleaned_df는 이미 'activity' 데이터를 포함하고 있음
data = cleaned_df
data['preferred'].fillna("hello", inplace=True)
data['requirement'].fillna("hello", inplace=True)# 'preferred' 컬럼에 누락된 값이 있을 경우 기본값으로 채움

# IT 포트폴리오 데이터 읽어오기
it_portfolio_df = pd.read_csv('/Users/jhkim97/Downloads/userid_1_filtered_correct_data.csv')  # 파일 경로 수정 필요

# 모델 및 토크나이저 불러오기
model_name = "jjzha/jobspanbert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# IT 포트폴리오 데이터를 기반으로 유사도 측정
for index, row in it_portfolio_df.iterrows():
    role_text = row['Role']
    project_title_text = row['ProjectTitle']
    technology_text = row['Technology']

    # IT 포트폴리오 데이터를 하나의 텍스트로 합치기
    it_portfolio_text = f"{role_text}. {project_title_text}. {technology_text}"

    # 포트폴리오 텍스트 토큰화 및 인덱싱
    user_inputs = tokenizer(it_portfolio_text, return_tensors="pt", padding=True, truncation=True)

    # activity 데이터를 처리하고 모델에 입력으로 전달
    combined_embeddings = []

    for idx, activity_row in data.iterrows():
        requirement_text = activity_row['requirement']
        preferred_text = activity_row['preferred']
        category_text = str(activity_row['category'])  # category_text가 단일 객체인 경우 문자열로 변환

        # 텍스트 토큰화 및 인덱싱
        requirement_inputs = tokenizer(requirement_text, return_tensors="pt", padding=True, truncation=True)
        preferred_inputs = tokenizer(preferred_text, return_tensors="pt", padding=True, truncation=True)
        category_inputs = tokenizer(category_text, return_tensors="pt", padding=True, truncation=True)

        # 모델에 전달하여 임베딩 얻기
        requirement_outputs = model(**requirement_inputs)
        preferred_outputs = model(**preferred_inputs)
        category_outputs = model(**category_inputs)

        # 임베딩을 합치기
        combined_requirement_embedding = requirement_outputs.last_hidden_state[:, 0, :]
        combined_preferred_embedding = preferred_outputs.last_hidden_state[:, 0, :]
        combined_category_embedding = category_outputs.last_hidden_state[:, 0, :]

        # 가중치를 적용하여 임베딩 결합
        weighted_combined_embedding = (0.4 * combined_requirement_embedding + 0.5 * combined_preferred_embedding + 0.3 * combined_category_embedding) / 1.2

        combined_embeddings.append(weighted_combined_embedding)

    # IT 포트폴리오 텍스트의 임베딩 얻기
    it_portfolio_embedding = model(**user_inputs).last_hidden_state[:, 0, :]

    # 각 요구사항, 선호사항 및 category와 IT 포트폴리오 간의 가중치 적용된 코사인 유사도 계산
    similarities = [1 - cosine(it_portfolio_embedding.detach().numpy().flatten(), weighted_combined_embedding.detach().numpy().flatten()) for weighted_combined_embedding in combined_embeddings]

    # 유사도를 기준으로 상위 10개 활동을 저장
    top_10_indices = sorted(range(len(similarities)), key=lambda i: similarities[i], reverse=True)[:10]
    top_10_activities = [(activity_index, similarities[activity_index]) for activity_index in top_10_indices]
    top_10_similarities.append((it_portfolio_text, top_10_activities))

# 상위 10개
    # 상위 10개 유사도를 출력
    for idx, (portfolio_text, top_10_activities) in enumerate(top_10_similarities):
        print(f"IT Portfolio {idx + 1}:")
    for activity_index, similarity in top_10_activities:
        print(f"  - Activity {activity_index + 273}, Similarity: {similarity:.4f}")

Some weights of the model checkpoint at jjzha/jobspanbert-base-cased were not used when initializing BertModel: ['cls.span_predictions.query_end_transform.LayerNorm.bias', 'cls.span_predictions.end_transform.LayerNorm.weight', 'cls.span_predictions.end_transform.dense.weight', 'cls.span_predictions.start_transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.span_predictions.end_transform.dense.bias', 'cls.span_predictions.end_classifier', 'cls.span_predictions.query_end_transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.span_predictions.query_start_transform.dense.bias', 'cls.span_predictions.query_start_transform.LayerNorm.bias', 'cls.span_predictions.end_transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.span_predictions.start_classifier', 'cls.predictions.decoder.weight', 'cls.span_predictions.query_end_transform.dense.weight', 'cls.span_predictions.start_transform

IT Portfolio 1:
  - Activity 275, Similarity: 0.7595
  - Activity 294, Similarity: 0.7254
  - Activity 298, Similarity: 0.7234
  - Activity 274, Similarity: 0.7225
  - Activity 276, Similarity: 0.7109
  - Activity 285, Similarity: 0.6991
  - Activity 288, Similarity: 0.6947
  - Activity 284, Similarity: 0.6945
  - Activity 289, Similarity: 0.6912
  - Activity 290, Similarity: 0.6889
